In [1]:
import pandas as pd
import numpy as np
import re
import json
import ollama
import chromadb
import time

=============================================
# Pipeline functions
=============================================

In [14]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)

    def interpret_brightness(brightness_level):
        if 0 <= brightness_level <= 100:
            return "dark"
        elif 100 < brightness_level <= 150:
            return "dim"
        elif 150 < brightness_level <= 500:
            return "normal lighting"
        elif 500 < brightness_level <= 1000:
            return "very bright"
        else:
            return "dark"

    def generate_description(row):
        brightness_description = interpret_brightness(row['BrightnessLevel'])
        # return f"{row['UserActivity']} in the {row['Location']} when it is {brightness_description} at {row['Timestamp']}."
        pd.set_option('display.max_colwidth', None)
        description = f"{row['UserActivity']} in the {row['Location']} at {row['Timestamp']} when it is {brightness_description}."
        return description

    df['Description'] = df.apply(generate_description, axis=1)

    # only use the lastest state
    df_latest = df.iloc[-1:]
    df_latest = df_latest['Description']
    df_latest = df_latest.to_string(index=False)
    # print(df_latest)

    # use last 10 states
    df_history = df.iloc[-10:]
    df_history = df_history['Description']
    df_history = df_history.to_string(index=False)
    # print(df_history)

    return df_latest, df_history


    
def LLM2(knowledge_base, df_latest, collection, df_history):

    # user_query = f"""
    # There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    # Tell me Which lights should be turned on/off while the user is {df_latest}? "
    # """ 

    user_query = f"""
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. light5 is the main light in the dining room next to living room closely.
    recent user behaviors are: {df_history}tell me Which lights should be turned on/off while the user is {df_latest}? "
    """ 

    print("User Query:")
    print(user_query)

    query_response = ollama.embeddings(
        prompt=user_query,
        model="mxbai-embed-large"
    )

    results = collection.query(
        query_embeddings=[query_response["embedding"]],
        n_results= 10  # Retrieve top 10 relevant documents
    )

    retrieved_documents = results['documents'][0]  
    data = ' '.join(retrieved_documents)
    print("Retrieved Data:")
    print(data)

    final_prompt = f"""
    You are a smart home assistant.

    Based on the knowledge base entries:
    {data}

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
        "description": "{df_latest}",
        "light1": "on/off",
        "light2": "on/off",
        "light3": "on/off",
        "light4": "open/close",
        "light5": "on/off",

    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    actions = output['response']

    print("Assistant's Response:")
    print(actions)

    return actions



def LLM1(df_latest, user_feedback, actions, df_history, collection):
    print("User Feedback:")
    print(user_feedback)
    
    user_update = f"""
    You are a smart home assistant responsible for deciding if to update the lighting knowledge based on user manually changes to the light states. 
    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, there are some trasition phase, so decide if to update the knowledge base based on the user feedback and recent behaviors.
    """

    query_response = ollama.embeddings(
        prompt=user_update,
        model="mxbai-embed-large"
    )

    results = collection.query(
        query_embeddings=[query_response["embedding"]],
        n_results=8  # Retrieve top 10 relevant documents
    )

    retrieved_documents = results['documents'][0]  
    data = ' '.join(retrieved_documents)
    print("Retrieved Data:")
    print(data)

    final_prompt_update = f"""
    You are a smart home assistant.

    Based on the retrieved current knowledge base entries:
    {data}

    Consider current situation:
    {user_update}

    Decide if to update the knowledge base in the fixed json format:
    {{
    "Update": "yes/no"  
    }}

    Provide the updated knowledge base in the fixed json format:
    {{
        "{df_latest}:",
        "light1": "on/off",
        "light2": "on/off",
        "light3": "on/off",
        "light4": "open/close",
        "light5": "on/off",

    }}
    """

    ## LLM1
    output = ollama.generate(
    model="qwen2.5:14b-instruct",
    prompt=final_prompt_update
    )

    knowledge_base_text = output['response']
    # print(knowledge_base_text)
    return knowledge_base_text






def update_knowledge_base(knowledge_base, knowledge_base_update):
    # Define patterns to extract JSON code blocks
    code_block_pattern = r'```json\s*([\s\S]*?)```'
    json_blocks = re.findall(code_block_pattern, knowledge_base_update)

    # If no code blocks are found, try to extract any JSON-like strings
    if not json_blocks:
        # Try to find any JSON objects in the text
        json_blocks = re.findall(r'\{(?:[^{}]|(?R))*\}', knowledge_base_update)

    # Initialize variables to store the extracted data
    update_decision = None
    updated_kb = None

    for idx, json_str in enumerate(json_blocks):
        print(f"\nExtracted JSON String {idx+1}:")
        print(json_str)

        # Clean up the JSON string
        json_str = json_str.strip()
        json_str = re.sub(r',\s*}', '}', json_str)  # Remove trailing commas before }
        json_str = re.sub(r',\s*\]', ']', json_str)  # Remove trailing commas before ]

        # Parse the JSON string
        try:
            json_data = json.loads(json_str)

            # Check if the JSON data contains "Update" key
            if "Update" in json_data:
                update_decision = json_data.get("Update")
                print(f"\nParsed Update Decision: {update_decision}")

                # Remove "Update" from json_data to get the updated knowledge base
                json_data.pop("Update", None)

                # If there are other keys, they represent the updated knowledge base
                if json_data:
                    updated_kb = json_data
                    print("\nParsed Updated Knowledge Base:")
                    print(json.dumps(updated_kb, indent=2))
            else:
                # If "Update" is not present, assume this is the updated knowledge base
                updated_kb = json_data
                print("\nParsed Updated Knowledge Base:")
                print(json.dumps(updated_kb, indent=2))
        except json.JSONDecodeError as e:
            print(f"\nFailed to parse JSON in block {idx+1}: {e}")

    # Process the extracted data
    if update_decision is not None:
        print(f"\nUpdate Decision: {update_decision}")
        if update_decision.lower() == "yes":
            if updated_kb is not None:
                # Update the knowledge base
                knowledge_base = knowledge_base + "\n" + json.dumps(updated_kb, indent=2)
                print("\nKnowledge base has been updated.")
            else:
                print("\nNo updated knowledge base provided.")
        else:
            print("\nKnowledge base remains unchanged.")
    else:
        print("\nThe 'Update' decision was not found in the LLM's output.")

    return knowledge_base


def get_user_confidence():
    confidence_input = input("Please rate your confidence in the system on a scale from 0 to 1: ")
    try:
        confidence = float(confidence_input)
        if 0 <= confidence <= 1:
            return confidence
        else:
            print("Invalid input. Confidence should be between 0 and 1.")
            return get_user_confidence()
    except ValueError:
        print("Invalid input. Please enter a number between 0 and 1.")
        return get_user_confidence()
    

knowledge_base = """
    This is the preference of the user:
    Prefer a dimly lit while watching TV.
    Prefer a dimly lit environment while reading books.
    Prefer a bright environment while exercising.
    Prefer a bright environment while eating.
    Prefer a dimly lit environment while playing video games.
    Prefer a dark environment while lying. 
    Prefer a bright environment while exercising.
    Prefer a bright environment while housekeeping.
    Prefer a dimly lit environment while typing.
    Prefer a dimly lit environment while sitting.
    Prefer a bright environment while cooking.
    Prefer a bright environment while grooming.
"""

In [5]:
knowledge_base_json = """
{
  "Layout of lights in the living room": [
    {
      "light_id": "light1",
      "description": "Main light with 3 color temperatures: warm, neutral, and cool."
    },
    {
      "light_id": "light2",
      "description": "Dimming light above the TV."
    },
    {
      "light_id": "light3",
      "description": "Lamp on the table."
    },
    {
      "light_id": "light4",
      "description": "Curtain on the window."
    },
    {
      "light_id": "light5",
      "description": "Main light in the dining room next to the living room."
    }
    
  ],
  "users": [
    {
      "user_id": "user1",
      "name": "Richard",
      "preferences": [
        {
          "activity": "watching TV",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "reading",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "cooking",
          "preferred_lighting": "bright"
        },
        {
          "activity": "sitting",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "lying",
          "preferred_lighting": "dark"
        },
        {
          "activity": "typing",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "housekeeping",
          "preferred_lighting": "bright"
        },
        {
          "activity": "exercising",
          "preferred_lighting": "bright"
        },
        {
          "activity": "video gaming",
          "preferred_lighting": "dimly lit"
        },
        {
            "activity": "eating",
            "preferred_lighting": "bright"
        },
        {
            "activity": "grooming",
            "preferred_lighting": "bright"
        }
      ]
    },
    {
      "user_id": "user2",
      "name": "Di",
      "preferences": [
        {
          "activity": "watching TV",
          "preferred_lighting": "dark"
        },
        {
          "activity": "reading",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "cooking",
          "preferred_lighting": "bright"
        },
        {
          "activity": "sitting",
          "preferred_lighting": "bright"
        },
        {
          "activity": "lying",
          "preferred_lighting": "dark"
        },
        {
          "activity": "typing",
          "preferred_lighting": "bright"
        },
        {
          "activity": "housekeeping",
          "preferred_lighting": "bright"
        },
        {
          "activity": "exercising",
          "preferred_lighting": "bright"
        },
        {
          "activity": "video gaming",
          "preferred_lighting": "dark"
        },
        {
            "activity": "eating",
            "preferred_lighting": "bright"
        },
        {
            "activity": "grooming",
            "preferred_lighting": "dimly lit"
        }
      ]
    },
    {
      "user_id": "user3",
      "name": "Mingyi",
      "preferences": [
        {
          "activity": "watching TV",
          "preferred_lighting": "bright"
        },
        {
          "activity": "reading",
          "preferred_lighting": "bright"
        },
        {
          "activity": "cooking",
          "preferred_lighting": "bright"
        },
        {
          "activity": "sitting",
          "preferred_lighting": "bright"
        },
        {
          "activity": "lying",
          "preferred_lighting": "dark"
        },
        {
          "activity": "typing",
          "preferred_lighting": "bright"
        },
        {
          "activity": "housekeeping",
          "preferred_lighting": "bright"
        },
        {
          "activity": "exercising",
          "preferred_lighting": "bright"
        },
        {
          "activity": "video gaming",
          "preferred_lighting": "dimly lit"
        },
        {
            "activity": "eating",
            "preferred_lighting": "bright"
        },
        {
            "activity": "grooming",
            "preferred_lighting": "bright"
        }
      ]
    }

  ]
}

"""

=============================================
# Pipeline Main
=============================================

In [16]:
documents = knowledge_base.split('\n\n')
documents = [' '.join(doc.split()) for doc in documents]

client = chromadb.Client()

try:
    collection = client.get_collection("docs")
    print("Collection already exists.")
except Exception as e:
    # If the collection does not exist, create it
    collection = client.create_collection(name="docs")
    print("Collection created.")

# store each document in a vector embedding database
for i, d in enumerate(documents):
    response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
    embedding = response["embedding"]
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[d]
    )


def main():
    global knowledge_base
    previous_df_latest = None

    user = "Richard"

    while True:
        df_latest, df_history = load_data('simulated_data/reading.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = LLM2(knowledge_base, df_latest, collection, df_history)
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "close", "light5": "off"}
            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            if user_feedback:
                user_condidence = get_user_confidence()
                # if user_condidence < 0.5:
                #     # use RL agent to update the knowledge base
                #     print("RL agent to update the knowledge base")
                #     knowledge_base = rl_agent_update(knowledge_base, user_feedback, actions, df_history)

                # else:
                #     knowledge_base_update = LLM1(user_feedback, actions, df_history)
                #     knowledge_base = update_knowledge_base(knowledge_base, knowledge_base_update)
                #     print("\nUpdated Knowledge Base:")
                #     print(knowledge_base)
                print(user_feedback)
                # knowledge_base_update = LLM1(user_feedback, actions, df_history, collection)
                knowledge_base_update = LLM1(
                    df_latest=df_latest,
                    user_feedback=user_feedback,
                    actions=actions,
                    df_history=df_history,
                    collection=collection)
                
                knowledge_base = update_knowledge_base(knowledge_base, knowledge_base_update)
                print("\nUpdated Knowledge Base:")
                print(knowledge_base)


            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main()

Collection already exists.


Add of existing embedding ID: 0
Insert of existing embedding ID: 0


description changed
housekeeping in the living room at 14:30:00 when it is very bright.
User Query:

    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. light5 is the main light in the dining room next to living room closely.
    tell me Which lights should be turned on/off while the user is housekeeping in the living room at 14:30:00 when it is very bright.? "
    
Retrieved Data:
Prefer a bright environment while housekeeping. Prefer a bright environment while cooking. Prefer a bright environment while eating. Prefer a bright environment while grooming. Prefer a dimly lit while watching TV. Prefer a dimly lit environment while sitting. Prefer a dark environment while lying. Prefer a bright environment while exercising. Prefer a bright environment while exercising. Prefer a dimly lit environment while typing.
Ass

KeyboardInterrupt: 

In [12]:
print(knowledge_base)


    This is the preference of the user:
    Prefer a dimly lit while watching TV.
    Prefer a dimly lit environment while reading books.
    Prefer a bright environment while exercising.
    Prefer a bright environment while eating.
    Prefer a dimly lit environment while playing video games.
    Prefer a dark environment while lying. 
    Prefer a bright environment while exercising.
    Prefer a bright environment while housekeeping.
    Prefer a dimly lit environment while typing.
    Prefer a dimly lit environment while sitting.
    Prefer a bright environment while cooking.
    Prefer a bright environment while grooming.



=============================================
# 0-Shot
=============================================

In [ ]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def zero_shot(user, df_latest, user_feedback, df_history, actions):
    print("User Feedback:")
    print(user_feedback)
    
    user_query = f"""
    {user} is the user who is living in this apartment. 

    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, decide if to update the lights base based on the user feedback.
    """

    final_prompt = f"""
    You are a smart home assistant.

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "lightx": "on/off",
        "lightx": "on/off",
        ...
    }},
    ...
    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    actions = output['response']

    print("Assistant's Response:")
    print(actions)

    return actions

def main_zeroshot():
    global knowledge_base
    previous_df_latest = None
    user = "Richard"

    while True:
        df_latest, df_history = load_data('simulated_data.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = zero_shot(
                user = user,
                df_latest = df_latest,
                user_feedback = None,
                df_history = None,
                actions = None)
            
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "open", "light5": "off"}
        
            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            actions_feedback = zero_shot(
                user=user,
                df_latest=df_latest,
                user_feedback=user_feedback,
                df_history=df_history,
                actions=actions)
            
            print("\nUpdated Action:")
            print(actions_feedback)

            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_zeroshot()

=============================================
# Chain of Thought
=============================================

In [ ]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def cot(user, df_latest, user_feedback, df_history, actions):
    print("User Feedback:")
    print(user_feedback)
    
    user_query = f"""
    {user} is the user who is living in this apartment.
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, decide if to update the lights base based on the user feedback.

    Provide your reasoning step by step.
    """

    final_prompt = f"""
    You are a smart home assistant.

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "lightx": "on/off",
        "lightx": "on/off",
        ...
    }},
    ...
    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    actions = output['response']

    print("Assistant's Response:")
    print(actions)

    return actions

def main_cot():
    global knowledge_base
    previous_df_latest = None
    user =  "Richard"

    while True:
        df_latest, df_history = load_data('simulated_data.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = cot(
                user = user,
                df_latest = df_latest,
                user_feedback = None,
                df_history = None,
                actions = None)
            
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "open", "light5": "off"}
        

            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            actions_feedback = cot(
                user=user,
                df_latest=df_latest,
                user_feedback=user_feedback,
                df_history=df_history,
                actions=actions)
            
            print("\nUpdated Action:")
            print(actions_feedback)

            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_cot()

=============================================
# ReACT
=============================================

In [4]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def react(user, df_latest, user_feedback, df_history, actions):
    print("User Feedback:")
    print(user_feedback)
    
    user_query = f"""

    {user} is the user who is living in this apartment.
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, decide if to update the lights base based on the user feedback.

    
    Determine which lights should be turned on/off while the user is **{df_latest}**, considering the user preferences, lighting setup, user feedback, and recent user behaviors.
    For each step, first provide your reasoning, then specify any action you would take (e.g., "Action: [describe action]").
    If you need to recall user preferences or recent behaviors, indicate that you are accessing that information.
    After reasoning and actions, provide the final light control actions in the fixed format.
    """

    final_prompt = f"""
    You are a smart home assistant.

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "lightx": "on/off",
        "lightx": "on/off",
        ...
    }},
    ...
    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    actions = output['response']

    print("Assistant's Response:")
    print(actions)

    return actions

def main_react():
    global knowledge_base
    previous_df_latest = None
    user = "Richard"

    while True:
        df_latest, df_history = load_data('simulated_data.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = react(
                user = user,
                df_latest = df_latest,
                user_feedback = None,
                df_history = None,
                actions = None)
            
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "open", "light5": "off"}
        

            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            actions_feedback = react(
                user=user,
                df_latest=df_latest,
                user_feedback=user_feedback,
                df_history=df_history,
                actions=actions)
            
            print("\nUpdated Action:")
            print(actions_feedback)

            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_react()

description changed
reading on the sofa at 11:04:00 when it is very bright.
User Feedback:
None
Assistant's Response:
Given that Richard is reading on the sofa at 11:04 PM and it's stated to be very bright, the primary concern would be to create a comfortable reading environment without overwhelming brightness. Since he’s reading, dimmer lighting would be preferable.

- Light1 (main living room light): Given its multiple color temperatures, setting it to a warmer tone might be ideal for evening ambiance.
- Light2 (dimming light above TV): This can provide additional focused illumination on the sofa area if needed without being too harsh.
- Light3 (lamp on table): A direct reading light next to where Richard is sitting would help and reduce strain on his eyes.

Light4 and Light5 are not directly relevant for this specific activity, so they should remain off unless requested otherwise by the user. Given it's very bright outside, curtains (light4) might be unnecessary if natural light isn

KeyboardInterrupt: 

=============================================
# RAG
=============================================

In [6]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def rag(user, knowledge_base, df_latest, collection, df_history, actions, user_feedback):
    print("User Feedback:")
    print(user_feedback)

    user_query = f"""
    {user} is the user who is living in this apartment. Based on the Layout of lights in the living room, 
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Decide if to update the knowledge base based on the user feedback and recent behaviors.
    """

    print("User Query:")
    print(user_query)

    query_response = ollama.embeddings(
        prompt=user_query,
        model="mxbai-embed-large"
    )

    results = collection.query(
        query_embeddings=[query_response["embedding"]],
        n_results=10  # Retrieve top 3 relevant documents
    )

    retrieved_documents = results['documents'][0]  
    data = ' '.join(retrieved_documents)
    print("Retrieved Data:")
    print(data)

    final_prompt = f"""
    You are a smart home assistant.

    Based on the knowledge base entries:
    {data}

    Respond to the user's current situation, user feedback and recent user behaviors:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "lightx": "on/off",
        "lightx": "on/off",
        ...
    }},
    ...
    }}

    Decide if to update the knowledge base in the fixed json format:
    {{
    "Update": "yes/no"  
    }}

    Provide the updated knowledge base in the fixed json format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "lightx": "on/off",
        "lightx": "on/off",
        ...
    }},
    ...
    }}

    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    actions = output['response']

    print("Assistant's Response:")
    print(actions)

    return actions


def main_rag():
    global knowledge_base
    previous_df_latest = None

    actions = None
    user_feedback = None

    user = "Richard"

    while True:
        df_latest, df_history = load_data('simulated_data.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = rag(
                user = user,
                knowledge_base = knowledge_base,
                df_latest=df_latest,
                collection=collection,
                df_history=df_history,
                actions=actions,
                user_feedback=user_feedback)
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "on", "light3": "off", "light4": "open", "light5": "off"}
            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            if user_feedback:
                user_condidence = get_user_confidence()

                print(user_feedback)
                # knowledge_base_update = LLM1(user_feedback, actions, df_history, collection)
                knowledge_base_update = rag(
                    user=user,
                    df_latest=df_latest,
                    user_feedback=user_feedback,
                    actions=actions,
                    df_history=df_history)
                
                knowledge_base = update_knowledge_base(knowledge_base, knowledge_base_update)
                print("\nUpdated Knowledge Base:")
                print(knowledge_base)


            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_rag()